## Tuning Parquet files

For high performance, Parquet requires a good schema and physical design. Let's look at 

* Data types
* Encodings
* And compressions.

### Parquet data types

Parquet has a complete type system.

**Physical data types** are the binary data types that Parquet stores data in:

- `BOOLEAN`: 1 bit boolean
- `INT32`: 32 bit signed ints
- `INT64`: 64 bit signed ints
- `FLOAT`: IEEE 32-bit floating point values
- `DOUBLE`: IEEE 64-bit floating point values
- `BYTE_ARRAY`: arbitrarily long byte arrays
- `FIXED_LEN_BYTE_ARRAY`: fixed length byte arrays

**Logical data types** are additional interpretations on top of the physical data types for outside representation and validation:

Numeric Types:

- `INT(bitWidth, isSigned)`: Signed/unsigned integers (8, 16, 32, 64 bits)
  - `INT8`, `INT16`, `INT32`, `INT64` (signed)
  - `UINT8`, `UINT16`, `UINT32`, `UINT64` (unsigned)
- `DECIMAL(precision, scale)`: Fixed-point decimal numbers (mapped to INT32, INT64, FIXED_LEN_BYTE_ARRAY)

String Types:

- `STRING`: UTF-8 encoded character strings
- `ENUM`: Enumerated string values
- `UUID`: 128-bit universally unique identifiers
- `JSON`: JSON-encoded strings

Temporal Types:

- `DATE`: Calendar date (days since Unix epoch)
- `TIME`: Time of day with microsecond or nanosecond precision
  - With/without timezone
- `TIMESTAMP`: Instant in time with microsecond or nanosecond precision
  - With/without timezone (UTC or local)
- `INTERVAL`: Time duration

Binary Types:

- `BSON`: Binary JSON format

Nested/Complex Types:

- `LIST`: Ordered collection of elements
- `MAP`: Key-value pairs
- `STRUCT`: Record with named fields (nested columns)

Parquet libraries will typically infer a working data type from the input data, but, for example, JSON has a much more elementary type system and you may need to "help" with the interpretation. 

### Sorting order

Sorting data by frequently queried columns (like `source` and `time`) improves compression and enables predicate pushdown during queries. 

To improve compression, Parquet applies run-length encoding and can apply delta encoding:

* **Run-length encoding** means that repeating values are stored as a tuple of (number of repetitions, value). Conceptually, if your values are "device_1", "device_1", "device_2", "device_2", the run-length encoding is (2, "device_1"), (2, "device_2").
* **Delta encoding** means that the difference between two values is stored. For example, if you have data coming into the system every second for a device, instead of storing each timestamp, only the initial timestamp and a sequence of 1000 (milliseconds) is stored. This way, you get many similar values that can be efficiently run-length encoded.

Predicate pushdown means that certain parts of files can be skipped during queries. If you sort by device, you can ideally just read the sections of the files with that device. 

Let's first read again the original events as they are, and then sort them by "source" (device) and time.

In [ ]:
import daft
import pyarrow.parquet as pq
from pathlib import Path
from time import time
import matplotlib.pyplot as plt
import dateutil
import re

%reload_ext autoreload
%autoreload 2
from helpers import inspect, compare_sizes

events_data = daft.read_json('../data/input/events.jsonl').exclude('id')
files = events_data.write_parquet('../data/output/events')
events_parquet_path = Path(files.to_pydict()['path'][0])

events_parquet = pq.ParquetFile(events_parquet_path)
print(f"Base file size: {events_parquet_path.stat().st_size:,} bytes")

In [ ]:
print("Sorting by device and time...")
events_data_sorted = events_data.sort([daft.col('source'), daft.col('time')])

files = events_data_sorted.write_parquet('../data/output/events_sorted')
events_parquet_path_sorted = Path(files.to_pydict()['path'][0])

events_parquet_sorted = pq.ParquetFile(events_parquet_path_sorted)
compare_sizes(events_parquet, "Original", events_parquet_sorted, "Sorted")

How can you explain the size differences? Verify your explanation using `parqeye` and `parquet-tools`. For example, to inspect the `source` column in more detail, use

```
parquet-tools inspect --row-group 0 --column-chunk 2 ….parquet | python3 -mjson.tool 
parquet-tools inspect --row-group 0 --column-chunk 2 --page 0 ….parquet | python3 -mjson.tool 
```

What do you think, is delta encoding efficient on events? Why or why not?

Note: Page 0 usually holds the dictionary data, if a dictionary is present.

### Data modelling

In our example, events contain formatted log messages with a text and the ID of the work piece. 

```
Starting to work on workpiece 2024_9550021
Stop to work on workpiece 2024_9550021
```

This is a common approach, but for analytics, it would be much more efficient to separate the workpiece ID right away instead of first formatting it and then parsing it back. Let's split the log entry and add a column for the workpiece ID.

In [ ]:
events_data_split = events_data.with_columns({
    "workpiece_year": daft.col("text").regexp_extract(r"workpiece\s+(\d+)_", 1).cast(daft.DataType.int32()),
    "workpiece_num": daft.col("text").regexp_extract(r"workpiece\s+\d+_(\d+)", 1).cast(daft.DataType.int64()),
    "text": daft.col("text").regexp_replace(r"work\s+.*$", "work")
}).sort([daft.col('source'), daft.col('time')])

start_time = time()
files = events_data_split.write_parquet('../data/output/events_split')
events_parquet_path_split = Path(files.to_pydict()['path'][0])
write_time = time() - start_time

events_parquet_split = pq.ParquetFile(events_parquet_path_split)
compare_sizes(events_parquet_sorted, "Sorted", events_parquet_split, "Split")
print(f"Write time (Snappy): {write_time:.2f}s")

### Compression

gzip can be used instead of snappy as compression.

In [ ]:
start_time = time()
files = events_data_split.write_parquet('../data/output/events_gzip', compression="gzip")
events_parquet_path_gzip = Path(files.to_pydict()['path'][0])
gzip_time = time() - start_time

print(f"Snappy Size: {events_parquet_path_split.stat().st_size:,} bytes | Time: {write_time:.2f}s")
print(f"Gzip Size:   {events_parquet_path_gzip.stat().st_size:,} bytes | Time: {gzip_time:.2f}s")
print(f"Reduction vs Original: {events_parquet_path_gzip.stat().st_size/events_parquet_path.stat().st_size * 100:.2f}%")

Much better compression, but also quite some performance impact -- also for querying!

Note: If you use gzip on original JSON file, you will reach 23475269 bytes. So the optimized Parquet version is only 42% of the original file.

## Querying

Let's query the new data and visualize the weekly workpiece production.

Note: Daft does not yet support "date_trunc" in SQL, so this example uses Daft's lazy DataFrame API.

In [ ]:
result = (
    events_data_split
    .with_column("week", daft.col("time").date_trunc("1 week"))
    .with_column("workpiece_id",
        daft.col("workpiece_year").cast(daft.DataType.string()) + "_" +
        daft.col("workpiece_num").cast(daft.DataType.string()))
    .groupby("week")
    .agg(daft.col("workpiece_id").count_distinct().alias("pieces_per_week"))
    .sort("week")
    .to_pandas()
)

# Visualize
plt.figure(figsize=(12, 6))
plt.plot(result['week'], result['pieces_per_week'], marker='o', linewidth=2)
plt.xlabel('Week')
plt.ylabel('Workpieces')
plt.title('Weekly Workpiece Production')
plt.grid(True, alpha=0.3)
plt.show()

## Summary

Parquet has a rich type system and efficient encoding and compression methods. By default, reasonable types and compression methods are inferred from the source data. Howevever, sorting is an essential to both get optimized Parquet files and efficient querying through "predicate pushdown".

Other insights: 
* Parquet uses only a few physical types in the column storage layer; all other standard types are applied on top.
* "Snappy" compression seems a good tradeoff between disk and CPU resource usage.
* Parquet works best with larger files in the area of Gigabyes. Besides the correct implementation and encoding of data types, there are also a number of other tuning parameters, like row group and data page size. Parquet [recommends](https://parquet.apache.org/docs/file-format/configurations/) row groups at 1GB size and data page sizes at 8KB. Here are [some benchmarks](https://developer.nvidia.com/blog/encoding-and-compression-guide-for-parquet-string-data-using-rapids/).
* Parquet libraries differ in the quality of the Parquet files that they produce. PyArrow, for example, does a rather "quick and dirty" job as can be seen in a [previous version of the notebook](https://github.com/Cumulocity-IoT/knee-deep-in-the-lake/blob/bfdf6f0b4d6fe897247e6cb0ff83b26d79738363/01_parquet/02_tuning.ipynb). Daft and Java seem to be smarter out of the box. If you chose a library, verify the output using the Parquet inspection tools.

In the next section, we will cover the odds and ends of handling semi-structured data with Parquet.